**Remap Labels for Final Augmented Masks**

This notebook remaps class labels in the final synthetic masks to match the semantic segmentation setup used in training.

Original PhenoBench masks contain 5 labels:
- 0 = Background
- 1 = Crop
- 2 = Weed
- 3 = PartialCrop
- 4 = PartialWeed

For training purposes, we merge the partial labels:
- 3 → 1 (PartialCrop becomes Crop)
- 4 → 2 (PartialWeed becomes Weed)

In [ ]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

def remap_labels(input_dir, output_dir=None):
    if output_dir is None:
        output_dir = input_dir
    else:
        os.makedirs(output_dir, exist_ok=True)

    file_list = [f for f in os.listdir(input_dir) if f.endswith(".png")]

    for file_name in tqdm(file_list):
        img_path = os.path.join(input_dir, file_name)
        img = Image.open(img_path)
        img_array = np.array(img)

        img_array[img_array == 3] = 1  # partial-crop → crop
        img_array[img_array == 4] = 2  # partial-weed → weed

        new_img = Image.fromarray(img_array)
        new_img.save(os.path.join(output_dir, file_name))

input_dir = "final_masks"
output_dir = "final_masks_remapped"

remap_labels(input_dir, output_dir)

In [ ]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

def remap_subset_labels(subset_dir, semantics_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    subset_names = [os.path.splitext(f)[0] for f in os.listdir(subset_dir) if f.endswith(".png")]

    for name in tqdm(subset_names):
        mask_path = os.path.join(semantics_dir, f"{name}.png")
        if not os.path.exists(mask_path):
            continue

        img = Image.open(mask_path)
        img_array = np.array(img)

        img_array[img_array == 3] = 1
        img_array[img_array == 4] = 2

        out_path = os.path.join(output_dir, f"{name}.png")
        Image.fromarray(img_array).save(out_path)

remap_subset_labels(
    subset_dir="subset_crops",                         
    semantics_dir="semantics",                         
    output_dir="masks_remapped_subset"           
)

In [ ]:
import os

def remove_mask_suffix(folder):
    for filename in os.listdir(folder):
        if filename.endswith("_mask.png"):
            old_path = os.path.join(folder, filename)
            new_filename = filename.replace("_mask", "")
            new_path = os.path.join(folder, new_filename)
            os.rename(old_path, new_path)
            print(f"{filename} → {new_filename}")

remove_mask_suffix("final_masks_remapped")
remove_mask_suffix("masks_remapped_subset")
remove_mask_suffix("masks_remapped_weeds")